In [1]:
import pandas as pd
import re
import openpyxl
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Label
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.io import output_notebook
from bokeh.models import NumeralTickFormatter
import jdatetime
import pandas as pd
import random
import seaborn as sns
from datetime import datetime, timedelta
from convertdate import persian

## Retriving Excel

In [2]:
df = pd.concat(pd.read_excel("/Users/mohammad/Dropbox/XAV coffee works/XAV Revenue/Excels/1403/بهار 1403/03 - خرداد/سفارشات کالا خرداد 1403.xlsx", sheet_name=None), ignore_index=True)
df2 = pd.concat(pd.read_excel("/Users/mohammad/Dropbox/XAV coffee works/XAV Revenue/Excels/1403/بهار 1403/02 - اردیبهشت/سفارشات کالا اردیبهشت 1403.xlsx",sheet_name=None),ignore_index=True)
df= pd.concat([df, df2],ignore_index=True)

def persian_to_gregorian(year, month, day):
    return persian.to_gregorian(year, month, day)

def gregorian_to_persian(year, month, day):
    return persian.from_gregorian(year, month, day)

def generate_date_list(start, end):
    # Split the input dates
    start_year, start_month, start_day = map(int, start.split('/'))
    end_year, end_month, end_day = map(int, end.split('/'))

    # Convert Persian dates to Gregorian dates
    start_gregorian = persian_to_gregorian(start_year, start_month, start_day)
    end_gregorian = persian_to_gregorian(end_year, end_month, end_day)

    # Convert tuples to datetime objects
    start_date_gregorian = datetime(*start_gregorian)
    end_date_gregorian = datetime(*end_gregorian)

    # Generate date range
    date_list = []
    current_date = start_date_gregorian
    while current_date <= end_date_gregorian:
        # Convert current Gregorian date to Persian date
        persian_date = gregorian_to_persian(current_date.year, current_date.month, current_date.day)
        date_list.append(f"{persian_date[0]}/{persian_date[1]}/{persian_date[2]}")
        # Move to the next day
        current_date += timedelta(days=1)

    return date_list



df["ردیف"] = df["ردیف"].fillna(method="ffill")
df.dropna(subset=["نام محصول"], inplace=True)
df = df.set_index("ردیف")
df = df.reset_index()
for i in df.index.to_list()[:-1]:
    row  = df.loc[i,"ردیف"]
    next_row = df.loc[i+1,"ردیف"]
    if next_row == row:
        df.loc[i+1,"تاریخ سفارش"] = df.loc[i,"تاریخ سفارش"]
        df.loc[i+1,"نام"] = df.loc[i,"نام"]
        df.loc[i+1,"نام خانوادگی"] = df.loc[i,"نام خانوادگی"]
        df.loc[i+1,"نام کافه"] = df.loc[i,"نام کافه"]
        df.loc[i+1,"شهر"] = df.loc[i,"شهر"]
        df.loc[i+1,"کد پیگیری"] = df.loc[i,"کد پیگیری"]



def convert_date(string):
    splited = string.split(" ")
    day_string = splited[0]
    day = splited[-3]
    month_string = splited[-2]
    month = None
    if month_string == "فروردین":
        month = 1
    elif month_string == "اردیبهشت":
        month = 2
    elif month_string == "خرداد":
        month = 3
    elif month_string == "تیر":
        month = 4
    elif month_string == "مرداد":
        month = 5
    elif month_string == "شهریور":
        month = 6
    elif month_string == "مهر":
        month = 7

    elif month_string == "آبان":
        month = 8

    elif month_string == "آذر":
        month = 9

    elif month_string == "دی":
        month = 10

    elif month_string == "بهمن":
        month = 11

    elif month_string == "اسفند":
        month = 12
    year = splited[-1]

    convert_date = f"{year}/{month}/{day}"

    return convert_date

df.dropna(subset=["تاریخ سفارش"], inplace=True)
df.dropna(subset=["تعداد"], inplace=True)

df["تاریخ سفارش"]  = df["تاریخ سفارش"].apply(convert_date)

for i in df.index.to_list():
    try:
        count = int(df.loc[i,"تعداد"])
        df.loc[i,"واحد فرعی"] = df.loc[i,"واحد فرعی"] * count
    except:
        continue

start = str(input("start date of report :  "))
end = str(input("end date of report :  "))

start_day = int(start.split("/")[-1])
start_month = int(start.split("/")[-2])
year = int(start.split("/")[0])

end_day = int(end.split("/")[-1])
end_month = int(end.split("/")[-2])

date_list = []
if start_day <= end_day :
    # if both dates are in same month
    for i in range(start_day,end_day+1):
        date = str(year) + "/" + str(start_month) + "/" + str(i)
        date_list.append(date)

else:
    # first month dates
    for i in range(start_day,32):
        date =  str(year) + "/" + str(start_month) + "/" + str(i)
        date_list.append(date)
    
    # second month dates
    for i in range(1,end_day+1):
        date =  str(year) + "/" + str(end_month) + "/" + str(i)
        date_list.append(date)           
    
date_list

span_df =  df[df["تاریخ سفارش"].isin(date_list)]


# Deleting gifts and office use
rows_to_delete = []
for i in span_df.index.to_list():
    code = span_df.loc[i,"کد پیگیری"]
    name = span_df.loc[i,"نام"]
    if str(code)[:4] == "هدیه" or str(code)[:4] == "مصرف" or str(name)[:4] == "هدیه" or str(name)[:4] == "مصرف":
        rows_to_delete.append(i)

gifts_df = span_df.loc[rows_to_delete]
span_df = span_df.drop(rows_to_delete)


# whole sale 
total_sale = sum(span_df["واحد فرعی"].to_list())/1000



# Each Line Sale
def change_name(string):
    name = string.split("|")[-1]
    return name

span_df["نام محصول"] = span_df["نام محصول"].apply(change_name)

v_dict = {}
a_dict = {}
x_dict = {}
for i in list(span_df["نام محصول"].value_counts().keys()):
    a = span_df[span_df["نام محصول"] == i]
    kg =  round(sum(a["واحد فرعی"].to_list())/1000,3)
    code = str(span_df[span_df["نام محصول"] == i ]["کد محصول"].to_list()[0])[0]
    serie = None
    if code == "5":
        serie = "V"
        v_dict[i] = kg
    elif code == "3":
        serie = "A"
        a_dict[i] = kg
    
    elif code == "1":
        serie = "X"
        x_dict[i] = kg

# Each Cafe Buy
cafe_list = set(span_df["نام کافه"].to_list())
cafe_dict = {}
for i in cafe_list:
    buy = round(sum(span_df[span_df["نام کافه"]==i]["واحد فرعی"].to_list())/1000,3)
    cafe_dict[i] = {"kg": buy}

# Finding top 10 cafe
top_cafe = sorted(cafe_dict, key=lambda k: cafe_dict[k]['kg'], reverse=True)[:11]
top_cafe_dict = {}
for i in top_cafe:
    top_cafe_dict[i] = cafe_dict[i]["kg"]


# Each City Buy
city_list = set(span_df["شهر"].to_list())
city_dict = {}
for i in city_list:
    buy = round(sum(span_df[span_df["شهر"]==i]["واحد فرعی"].to_list())/1000,3)
    city_dict[i] = {"kg": buy}

# Finding top 10 cafe
top_city = sorted(city_dict, key=lambda k: city_dict[k]['kg'], reverse=True)[:11]
top_city_dict = {}
for i in top_city:
    top_city_dict[i] = city_dict[i]["kg"]


# plotting Total sale based on the series
total_sale = total_sale
v_series_sale = sum(v_dict.values())
a_series_sale = sum(a_dict.values())
x_series_sale = sum(x_dict.values())


plot_dict = {"X Series" : x_series_sale , "A Series":  a_series_sale, "V Series": v_series_sale, "Total": total_sale}

def each_line(dict1, string):  

    x = list(dict1.keys())
    y = list(dict1.values())

    # Create a ColumnDataSource
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(x))]
    source = ColumnDataSource(data=dict(x=x, y=y, color=colors))

    # Create the figure
    p = figure(x_range=x, height=600, width=1000 ,title=f"Each {string} Sale This Week",
            toolbar_location=None, tools="")

    # Render the bars
    p.vbar(x='x', top='y', width=0.5, source=source, color="color")
    

    # Customize the plot
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 45  # Adjust the angle of the x-axis labels
    if len(x) == 1:
        x_offset = 350
    elif len(x) == 2:
        x_offset = 220
    elif len(x) == 3:
        x_offset = 70
    elif len(x) == 4:
        x_offset = 110
    elif len(x) == 5:
        x_offset = 85
    elif len(x) ==  6 :
        x_offset = 70
    elif len(x) == 7:
        x_offset = 40
    elif len(x) == 8 :
        x_offset = 55
    elif len(x) == 9:
        x_offset= 40
    elif len(x)>10:
        x_offset=40

    for i, value in enumerate(y):
        label = Label(x=i, y=value, text=str(round(value, 2)), text_baseline='middle', text_align='center', text_font_size="8pt", x_offset=x_offset, y_offset=10)
        p.add_layout(label)

    # Show the plot
    output_notebook()
    show(p)

each_line(plot_dict,"Series")

# plotting Each Line of each series sale
each_line(v_dict, "V Series")
each_line(a_dict, "A Series")
each_line(x_dict, "X Series")
each_line(top_cafe_dict, "Cafe")
each_line(top_city_dict, "City")



/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XA

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [22]:
span_df



,ردیف,چک لیست خروج از دفتر,Unnamed: 1,چک لیست آماده سازی سفارشات,Unnamed: 3,چک لیست سفارشات کالا,Unnamed: 5,کد پیگیری,تاریخ سفارش,شماره فاکتور,...,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43
425,4.0,False,NaN,0.0,NaN,0.0,NaN,52806075701,1403/2/2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,4.0,False,NaN,0.0,NaN,0.0,NaN,52806075701,1403/2/2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,5.0,NaN,NaN,0.0,NaN,NaN,NaN,52807600501,1403/2/2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,5.0,NaN,NaN,0.0,NaN,NaN,NaN,52807600501,1403/2/2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,6.0,False,NaN,0.0,NaN,0.0,NaN,52809427301,1403/2/2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403/2/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403/2/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403/2/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403/2/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
sf = span_df[span_df["نام کافه"] == "کافه بیکری روستار آوا سنتر"]
# dict_1 = {}
# for i in set(sf["نام محصول"].to_list()):
#     gf = sum(sf[sf["نام محصول"]== i]["واحد فرعی"].to_list())
#     dict_1.update({i:gf})

# dict_1

{' Vortex': 64000.0, ' Hambella': 27500.0}

In [57]:
dict_1 = {}
cafe_list = set(span_df["نام کافه"].to_list())
for i in cafe_list:
    dict_1[i] = {}
    sf = span_df[span_df["نام کافه"] == i]
    for j in set(sf["نام محصول"].to_list()):
        gf = sum(sf[sf["نام محصول"]== j]["واحد فرعی"].to_list())/1000
        dict_1[i].update({j:gf})

cafe_df = pd.DataFrame(dict_1).T
cafe_df["Total"] = cafe_df.sum(axis=1)
cafe_df = cafe_df.sort_values(by='Total', ascending=False)




,Hambella,Milagros,Ratnagiri,Tamara,Peru Huancayo,Vortex,Hambella( Filter Roast),Soledad,Janson,Deborah Nirvana,Benjamin De Dianous,Elida Plano,Total
کافه بیکری روستار سانا سنتر,87.5,NaN,NaN,NaN,NaN,176.00,NaN,NaN,NaN,NaN,NaN,NaN,263.50
کافه بیکری روستار آوا سنتر,27.5,NaN,NaN,NaN,NaN,64.00,NaN,NaN,NaN,NaN,NaN,NaN,91.50
کافه دارچین,3.0,NaN,NaN,5.25,67.0,3.00,NaN,NaN,NaN,NaN,NaN,NaN,78.25
ویکافه,68.8,0.45,0.45,2.40,NaN,NaN,1.8,NaN,NaN,NaN,NaN,NaN,73.90
کافه کی آر بی,7.5,NaN,NaN,NaN,NaN,60.00,NaN,NaN,NaN,NaN,NaN,NaN,67.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
کافه بلوم,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15
کافه موچی,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,0.15
کافه کاپار,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15
کافه اریدو,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15


In [61]:
dict_2 = {}
city_list = set(span_df["شهر"].to_list())
for i in city_list:
    dict_2[i] = {}
    sf = span_df[span_df["شهر"] == i]
    for j in set(sf["نام محصول"].to_list()):
        gf = sum(sf[sf["نام محصول"]== j]["واحد فرعی"].to_list())/1000
        dict_2[i].update({j:gf})

city_df = pd.DataFrame(dict_2).T
city_df["Total"] = city_df.sum(axis=1)
city_df = city_df.sort_values(by='Total', ascending=False)
city_df

,Tamara,Hambella,Milagros,Ratnagiri,Hambella( Filter Roast),Deborah Nirvana,Benjamin De Dianous,Vortex,Elida Plano,Janson,Soledad,Peru Huancayo,Total
تهران,20.40,279.65,12.95,8.60,13.35,0.08,0.36,728.75,0.20,0.36,0.40,216.65,1281.75
البرز,5.70,31.50,3.60,2.70,0.45,NaN,NaN,62.00,NaN,NaN,NaN,68.80,174.75
قم,0.45,13.50,0.49,0.34,1.50,NaN,0.04,113.50,NaN,NaN,0.04,4.75,134.61
مشهد,0.30,3.00,0.45,0.45,0.30,NaN,NaN,80.00,NaN,NaN,NaN,1.35,85.85
مازندران,6.90,14.50,0.45,0.45,0.45,NaN,NaN,52.30,NaN,NaN,0.24,8.00,83.29
خرمشهر,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.00,NaN,NaN,NaN,NaN,60.00
کرمان,NaN,3.00,0.15,0.60,NaN,NaN,0.16,41.65,NaN,NaN,NaN,NaN,45.56
آذربایجان غربی,1.05,3.15,1.09,1.09,4.65,0.04,NaN,32.45,NaN,0.20,NaN,1.65,45.37
فارس,0.15,0.30,0.15,0.15,NaN,0.12,0.04,22.50,0.04,0.04,0.04,21.50,45.03
آذربایجان شرقی,0.75,4.05,0.30,0.15,0.30,NaN,NaN,30.95,NaN,NaN,0.08,4.80,41.38


In [13]:
from datetime import datetime, timedelta
from convertdate import persian

def persian_to_gregorian(year, month, day):
    return persian.to_gregorian(year, month, day)

def gregorian_to_persian(year, month, day):
    return persian.from_gregorian(year, month, day)

def generate_date_range(start_date, end_date):
    # Split the input dates
    start_year, start_month, start_day = map(int, start_date.split('/'))
    end_year, end_month, end_day = map(int, end_date.split('/'))

    # Convert Persian dates to Gregorian dates
    start_gregorian = persian_to_gregorian(start_year, start_month, start_day)
    end_gregorian = persian_to_gregorian(end_year, end_month, end_day)

    # Convert tuples to datetime objects
    start_date_gregorian = datetime(*start_gregorian)
    end_date_gregorian = datetime(*end_gregorian)

    # Generate date range
    date_range = []
    current_date = start_date_gregorian
    while current_date <= end_date_gregorian:
        # Convert current Gregorian date to Persian date
        persian_date = gregorian_to_persian(current_date.year, current_date.month, current_date.day)
        date_range.append(f"{persian_date[0]}/{persian_date[1]}/{persian_date[2]}")
        # Move to the next day
        current_date += timedelta(days=1)

    return date_range

# Example usage
start_date = "1402/2/2"
end_date = "1403/3/3"
date_range = generate_date_range(start_date, end_date)
for date in date_range:
    print(date)


1402/2/2
1402/2/3
1402/2/4
1402/2/5
1402/2/6
1402/2/7
1402/2/8
1402/2/9
1402/2/10
1402/2/11
1402/2/12
1402/2/13
1402/2/14
1402/2/15
1402/2/16
1402/2/17
1402/2/18
1402/2/19
1402/2/20
1402/2/21
1402/2/22
1402/2/23
1402/2/24
1402/2/25
1402/2/26
1402/2/27
1402/2/28
1402/2/29
1402/2/30
1402/2/31
1402/3/1
1402/3/2
1402/3/3
1402/3/4
1402/3/5
1402/3/6
1402/3/7
1402/3/8
1402/3/9
1402/3/10
1402/3/11
1402/3/12
1402/3/13
1402/3/14
1402/3/15
1402/3/16
1402/3/17
1402/3/18
1402/3/19
1402/3/20
1402/3/21
1402/3/22
1402/3/23
1402/3/24
1402/3/25
1402/3/26
1402/3/27
1402/3/28
1402/3/29
1402/3/30
1402/3/31
1402/4/1
1402/4/2
1402/4/3
1402/4/4
1402/4/5
1402/4/6
1402/4/7
1402/4/8
1402/4/9
1402/4/10
1402/4/11
1402/4/12
1402/4/13
1402/4/14
1402/4/15
1402/4/16
1402/4/17
1402/4/18
1402/4/19
1402/4/20
1402/4/21
1402/4/22
1402/4/23
1402/4/24
1402/4/25
1402/4/26
1402/4/27
1402/4/28
1402/4/29
1402/4/30
1402/4/31
1402/5/1
1402/5/2
1402/5/3
1402/5/4
1402/5/5
1402/5/6
1402/5/7
1402/5/8
1402/5/9
1402/5/10
1402/5/11
1402/

In [28]:
def convert_date(string):
    splited = string.split(" ")
    day = splited[-3]
    month_string = splited[-2]
    month = {"فروردین": 1, "اردیبهشت": 2, "خرداد": 3, "تیر": 4, "مرداد": 5, "شهریور": 6, "مهر": 7, "آبان": 8, "آذر": 9, "دی": 10, "بهمن": 11, "اسفند": 12}[month_string]
    year = splited[-1]
    return f"{year}/{month}/{day}"

# Function to clean the dataframe
def clean_dataframe(df):
    df["ردیف"] = df["ردیف"].fillna(method="ffill")
    df.dropna(subset=["نام محصول"], inplace=True)
    df = df.set_index("ردیف").reset_index()
    for i in df.index.to_list()[:-1]:
        row = df.loc[i, "ردیف"]
        next_row = df.loc[i+1, "ردیف"]
        if next_row == row:
            df.loc[i+1, ["تاریخ سفارش", "نام", "نام خانوادگی", "نام کافه", "شهر", "کد پیگیری"]] = df.loc[i, ["تاریخ سفارش", "نام", "نام خانوادگی", "نام کافه", "شهر", "کد پیگیری"]]
    return df

# Function to clean the dataframe based on date list
def filter_and_clean_dataframe(df, date_list):
    span_df = df[df["تاریخ سفارش"].isin(date_list)]
    rows_to_delete = [i for i in span_df.index.to_list() if str(span_df.loc[i, "کد پیگیری"])[:4] in ["هدیه", "مصرف"] or str(span_df.loc[i, "نام"])[:4] in ["هدیه", "مصرف"]]
    gifts_df = span_df.loc[rows_to_delete]
    span_df = span_df.drop(rows_to_delete)
    return span_df


df = pd.concat(pd.read_excel("/Users/mohammad/Dropbox/XAV coffee works/XAV Revenue/Excels/1403/بهار 1403/03 - خرداد/سفارشات کالا خرداد 1403.xlsx", sheet_name=None), ignore_index=True)

df = clean_dataframe(df)
df.dropna(subset=["تاریخ سفارش", "تعداد"], inplace=True)
for i in df.index.to_list():
    try:
        df.loc[i,"تاریخ سفارش"] = convert_date(df.loc[i,"تاریخ سفارش"])
    except:
        continue

df

/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XA

,ردیف,چک لیست خروج از دفتر,Unnamed: 1,چک لیست آماده سازی سفارشات,Unnamed: 3,چک لیست سفارشات کالا,Unnamed: 5,کد پیگیری,تاریخ سفارش,شماره فاکتور,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,فاکتور چاپی
0,1.0,False,NaN,0.0,NaN,0.0,NaN,273766661668,1403/2/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,False,NaN,0.0,NaN,0.0,NaN,273767268964,1403/2/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,False,NaN,0.0,NaN,0.0,NaN,273784764857,1403/2/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,False,NaN,0.0,NaN,0.0,NaN,273784764857,1403/2/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,0.0,NaN,NaN,NaN,273784757802,1403/2/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403/3/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1110,20.0,NaN,NaN,NaN,NaN,NaN,NaN,276598353253,1403/3/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1111,21.0,NaN,NaN,NaN,NaN,NaN,NaN,284979,1403/3/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+
1112,22.0,NaN,NaN,NaN,NaN,NaN,NaN,در انتظار پرداخت,1403/3/30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+


In [10]:
df

,چک لیست خروج از دفتر,Unnamed: 1,چک لیست آماده سازی سفارشات,Unnamed: 3,چک لیست سفارشات کالا,Unnamed: 5,ردیف,کد پیگیری,تاریخ سفارش,شماره فاکتور,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,فاکتور چاپی
0,False,NaN,0.0,NaN,0.0,NaN,1.0,273766661668,یکشنبه 30 اردیبهشت 1403,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,NaN,0.0,NaN,0.0,NaN,2.0,273767268964,یکشنبه 30 اردیبهشت 1403,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,NaN,0.0,NaN,0.0,NaN,3.0,273784764857,یکشنبه 30 اردیبهشت 1403,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.0,NaN,NaN,NaN,4.0,273784757802,یکشنبه 30 اردیبهشت 1403,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,NaN,NaN,NaN,10.0,NaN
6380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,NaN,NaN,NaN,28.0,NaN
6381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
